In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from futures.llm import LLM
from futures.futurist import Futurist
from futures.db import DB
import pandas as pd
from futures.futurist import Futurist
from futures.substack import SubstackManager
from collections import Counter
from langchain_openai import ChatOpenAI

In [ ]:
llm = LLM(datastore="data/")
sm = SubstackManager()
f = Futurist(llm)
db = DB(llm)
vectordb = db.createDB()

llm.ai = ChatOpenAI(model="gpt-4o-mini")

client=<openai.resources.embeddings.Embeddings object at 0x79b49a4b5750> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x79b49a55f5e0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002' openai_api_version=None openai_api_base=None openai_api_type=None openai_proxy=None embedding_ctx_length=8191 openai_api_key=SecretStr('**********') openai_organization=None allowed_special=None disallowed_special=None chunk_size=1000 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None http_async_client=None check_embedding_ctx_length=True
# VectorDB: continue on the DB
- VectorDB:  4091 elements already stored.
- VectorDB: adding 3252 documents.


In [57]:
db.getArticles(topic="Trump security", k =2)

[Document(id='f331a0bd-b990-4567-8690-7c08bcb9307a', metadata={'LEN': '9913', 'author': 'kelu', 'keywords': "['Supreme Court' 'ex-presidents' 'immunity' 'criminal prosecution'\n 'pre-election Trump trial' 'Washington criminal case' 'Donald Trump'\n 'presidential power' 'official acts' 'political divide']", 'origin': '20040714', 'source': 'https://apnews.com/article/supreme-court-trump-capitol-riot-immunity-2dc0d1c2368d404adc0054151490f542', 'src': '5cbf19bc5097424a9bd615c891be2a12', 'summary': 'The Supreme Court has ruled that former presidents have broad immunity from criminal prosecution for official acts, effectively delaying the Washington criminal case against ex-President Donald Trump. This ruling highlights a muscular view of presidential power, with the majority of conservative justices narrowing the case against Trump. This decision has sparked political divide, with dissenting judges criticizing it for undermining the principle that no person is above the law. The ruling also

# Read memory

In [14]:
meta = pd.read_parquet(llm.srcMetas).reset_index(drop=True)
signals = pd.read_parquet(llm.srcSeeds).reset_index(drop=True)
summaries = pd.read_parquet(llm.srcSummaries).reset_index(drop=True)
df = pd.read_parquet(llm.srcArticles)
df.columns = ["src","content","origin","LEN"]
techs = pd.read_parquet(llm.srcEmergingTechs).reset_index(drop=True)
behav = pd.read_parquet(llm.srcEmergingBehav).reset_index(drop=True)
issue = pd.read_parquet(llm.srcEmergingIssue).reset_index(drop=True)
cncrn = pd.read_parquet(llm.srcEmergingConcerns).reset_index(drop=True)

In [15]:
meta["path"] = meta["path"].apply(lambda row: row.split("/")[-1].split(".")[0].replace("d",""))
good = meta[meta["path"].str.startswith("2025")].src.tolist()
print(len(good),"articles in 2025")

1119 articles in 2025


In [23]:
themes = ", ".join(meta[meta.src.isin(good)].themes.tolist()).split(", ")
themes = [x.strip().lower() for x in themes]
counts = Counter(themes)

# Categorisation

In [37]:
S = "* "+"\n* ".join(signals[(signals.src.isin(good)) & (signals.relevancy > 4)].description)
categories  = llm.ai.invoke("You are a futurist. Read the list of signals below. You will have to find 20 categories and trends of change that cover these signals as much as possible:\n\n"+S).content
categories = categories.split("\n\n")[1:-1]
trends = {}
for cat in categories:
    title = cat.split("**  \n   ")[0][5:]
    desc = "\n".join(cat.split("**  \n   ")[1:])
    trends[title] = desc
twenty_trends = trends

In [ ]:
categories_tags = []
for ix, row in summaries.iterrows():
    if ix ==4000 :
        break
    txt = row.title + ": "+ row.summary

    #print(row.title)
    CATS = llm.ai.invoke("Read the text below, and find up to three trends to that this text illustrates. You can pick 0 to 3 trends.\
                   The categories are these: \n\n"+ \
                  "* "+"\n* ".join(list(twenty_trends.keys())) +"" + \
                  "\n\n"+txt+"\n\nYOU CAN ONLY PICK ONE TO THREE TRENDS, answer only with one line with trend with its name. Only the the names you have been given, and do not change them nor add to them. If no trend, answer None.").content
    CATS = CATS.replace("*","").split("\n")
    CATS = [x.strip() for x in CATS if len(x.strip())>0 and x.strip().lower() != "none"]
    categories_tags.append( (row.src, CATS) )
    #print(CATS)
categories_tags = pd.DataFrame(categories_tags, columns=["src","categories"])

In [64]:
categories_tags

,src,categories
0,b5f2219774c45faa79432dc8945c7638,"[Consumer Behavior and Values Shift, Culinary ..."
1,060e875dd9877f0d3555d38e682bae3d,[Technological Advancements in AI and Robotics...
2,94961e2c9317c22e226d4861726c131b,"[Consumer Behavior and Values Shift, Digital C..."
3,2c3e163f113838429299866828dad17c,[Technological Advancements in AI and Robotics...
4,c37bf0fae61cb13271841a3a22dfd018,[Cultural and Historical Narratives]
...,...,...
295,bbfdfdeb5b50e8e7068066eff334c9a2,"[Cultural and Historical Narratives, Digital C..."
296,a736d34eb783ad68b7ebf6d16e4b07cc,"[Digital Culture and Media Consumption, Cultur..."
297,8d86990465b681499154a10f6469520b,[Personal and Collective Identity]
298,ae01ccf94a3a28dbfdb21ac03f1cfae4,[Technological Advancements in AI and Robotics...
